### Manual Net New

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import os
import s3fs
import boto3
from io import BytesIO as bo
import gc
import polars as pl

In [ ]:
QTR_NET = 'Q3'
QTR = '2024Q3'

# Frozen Xponent Data week+7
CUR_WK = '20240802'
# Frozen Xponent Data week
PRE_WK = '20240726'
# PUT 1 if its the first week (the week after data frozne week) , else put 0
WEEK1 = 0
bucket = 'vortex-staging-a65ced90'

CUT_OFF_net = pd.Timestamp('2024-10-1')
#*THE 1ST DAY OF NEXT QUARTER, BASED QTR WE ARE GENERATING FILE FOR;

In [17]:
# Current Week Full Master Profile -
CUR_PROFILE = pd.read_parquet(f's3://{bucket}/PYADM/weekly/archive/{CUR_WK}/reference/MASTER_PROFILE.parquet')
# Previous Week Master Profile -
PRE_PROFILE = pd.read_parquet(f's3://{bucket}/PYADM/weekly/archive/{PRE_WK}/reference/MASTER_PROFILE.parquet')

In [18]:
# Perform the merge using pd.merge
NET_NEW = pd.merge(CUR_PROFILE, PRE_PROFILE['IID'],on='IID', how='left', indicator=True)
NET_NEW = NET_NEW[NET_NEW['_merge'] == 'left_only']
NET_NEW = NET_NEW.drop(['_merge'], axis=1).reset_index(drop=True)

#adding week timestamp 
NET_NEW['SOURCE'] = CUR_WK  #using cur_wk_xpn_date but in code  -"&CUR_WK." is used.

### Appending net new to create rolling file
- the week after frozen week , the net new file is reset
- otherwise, net new files gets accumilated by using previous week net new

In [21]:
NET_NEW_ACCU = pd.DataFrame()

net_new_varname = "NET_NEW_{}".format(QTR_NET)

if WEEK1 == 1:
    NET_NEW_ACCU = NET_NEW.copy()
elif WEEK1 == 0:
    globals()[net_new_varname] = pd.read_parquet(f's3://{bucket}/PYADM/weekly/archive/{PRE_WK}/reference/{net_new_varname}.parquet')
    columns_to_drop = ['Territory_IW1', 'Territory_Name_IW1','Region', 
                   'Region_Name', 'Area','Area_Name','SPECIALTY_DESCRIPTION',
                   'SPEC_INCL_LIN','IC_INCL_LIN'] 
    globals()[net_new_varname] = globals()[net_new_varname].drop(columns=columns_to_drop)
    NET_NEW_ACCU = pd.concat([NET_NEW, globals()[net_new_varname]], axis=0)

#### Assigning Terr information from Zip to Terr
- Addtionally, using the quarterly Speciality Inclusion Exclusion list to get inclusion status

In [26]:
ZIP_TO_TERR = pd.read_parquet(f's3://vortex-staging-a65ced90/PYADM/quaterly/{QTR}/geography/zip_to_terr.parquet')
ZIP_TO_TERR.rename(columns={'Territory':'Territory_IW1','Territory_Name':'Territory_Name_IW1'},inplace=True) #DOUBT not sure why we do this
SPEC_INCL_LIN = pd.read_parquet(f's3://vortex-staging-a65ced90/PYADM/quaterly/{QTR}/reference/qtrspec_SPEC_INCL_LIN.parquet')
#Extra
SPEC_INCL_LIN.rename(columns={'SPECIALTY_CD':'SpecialtyCode'},inplace=True)
SPEC_INCL_LIN.set_index('SpecialtyCode', inplace=True)

In [27]:
#getting terr name, region name, area name from zip to terr
NET_NEW_FN = pd.merge(NET_NEW_ACCU,ZIP_TO_TERR,left_on='ZIP',right_on='Zip',how='left')
whitespace_values = {
    'Territory_IW1': "1111-99999-99",
    'Territory_Name_IW1': "",
    'Region': "1111-999",
    'Region_Name': "",
    'Area': "1111-9",
    'Area_Name': ""
}
# Fill whitespace_values with the default values
NET_NEW_FN.fillna(whitespace_values, inplace=True)

In [28]:
#getting speciality flags from spec incl lin quaterly file -[NOTE : BUG FIX SUMMARY - sas did not perform a left join, revised code to perfrom mapping strategy] 
NET_NEW_FN['SPECIALTY_DESCRIPTION'] = NET_NEW_FN['SpecialtyCode'].map(SPEC_INCL_LIN['SPECIALTY_DESCRIPTION'])
NET_NEW_FN['SPEC_INCL_LIN'] = NET_NEW_FN['SpecialtyCode'].map(SPEC_INCL_LIN['SPEC_INCL'])
NET_NEW_FN.fillna({'SPECIALTY_DESCRIPTION':''},inplace=True)

In [29]:
#adding new column SPEC_INCL_LIN (this is based on match code, so ill have to wait for it to come.- NOTE  Condition is only applied on failed left matches
#NET_NEW_FN['SPEC_INCL_LIN'] = ['N' if x == '01' else 'Y' for x in NET_NEW_FN['MatchCode']] - This is wrong
NET_NEW_FN['SPEC_INCL_LIN'] = np.where(NET_NEW_FN['SPEC_INCL_LIN'].isna(),
                                        np.where(NET_NEW_FN['MatchCode'] == '01', 'N', 'Y'),
                                        NET_NEW_FN['SPEC_INCL_LIN'])

In [30]:
#dropping extra columns (different name , same data)
NET_NEW_FN.drop(['Zip'], axis=1, inplace=True)

### Filtering for any legal removal and unknown address HCPs
- 'Jami' file is used here (double check import location and check for file updates on quarter change)

In [31]:
# applying exclusions
JAMI_INCLEXCL = pd.read_parquet(f's3://vortex-staging-a65ced90/PYADM/reference/{QTR}/jami_inclexcl.parquet')
jami_exclusions = pd.DataFrame()
jami_exclusions = JAMI_INCLEXCL[JAMI_INCLEXCL['TYPE'].isin(['Legal Removals', 'Unknown Address'])]
jami_exclusions = jami_exclusions[['IID']].copy()
jami_exclusions.drop_duplicates(subset='IID', inplace=True)
jami_exclusions.reset_index(drop=True, inplace=True)

ref_week_NET_NEW = pd.merge(NET_NEW_FN,jami_exclusions,on = 'IID',how='left',indicator=True)
ref_week_NET_NEW = ref_week_NET_NEW[ref_week_NET_NEW['_merge'] == 'left_only']
ref_week_NET_NEW = ref_week_NET_NEW.drop(['_merge'], axis=1).reset_index(drop=True)

#### IC flag creation
- IC_INCL_LIN is a an important flag which decides if a hcp should be considered for IC or not
- CustomerEffectiveStartDate , SPEC_INCL_LIN, CustomerStatusCode, MatchCode decide its value

In [39]:
ref_week_NET_NEW['IC_INCL_LIN'] = 0  # Initialize the column with 0

condition = (
    (ref_week_NET_NEW['CustomerEffectiveStartDate'] < CUT_OFF_net) &
    (ref_week_NET_NEW['SPEC_INCL_LIN'] == "Y") &
    (ref_week_NET_NEW['CustomerStatusCode'] == "Active") &
    (ref_week_NET_NEW['MatchCode'] != '01')
)

ref_week_NET_NEW.loc[condition, 'IC_INCL_LIN'] = 1
print(ref_week_NET_NEW.shape)


(12607, 82)


Importing IIDs from frozen IC universe , and deleting any occurances of those in net new as an added QC

In [41]:
CUSTOMER_MASTER_IC_LIN = pd.read_parquet(f's3://vortex-staging-a65ced90/PYADM/quaterly/{QTR}/reference/CUSTOMER_MASTER_IC_LIN.parquet')
CUSTOMER_MASTER_IC_LIN = pd.DataFrame(CUSTOMER_MASTER_IC_LIN['IID'])
#Faulty Approach - needs checking into : Try using - ref_week_NET_NEW = merged_df[merged_df['_merge'] == 'left_only']
# incase of memory issues ?
# merged_df = pd.merge(ref_week_NET_NEW, CUSTOMER_MASTER_IC_LIN[['IID']], on='IID', how='inner')
# ref_week_NET_NEW = ref_week_NET_NEW.drop(merged_df.index)
#updated approach - 
qc_bef = len(ref_week_NET_NEW)
ref_week_NET_NEW = ref_week_NET_NEW[~ref_week_NET_NEW['IID'].isin(CUSTOMER_MASTER_IC_LIN['IID'])]
qc_af = len(ref_week_NET_NEW)
print("Records Dropped from ic uni: ",qc_bef-qc_af)

Records Dropped from ic uni:  0


In [42]:
#Only Keep the first Instance of IID Source.
qc_bef = len(ref_week_NET_NEW)
ref_week_NET_NEW.sort_values(by=['IID', 'SOURCE'], inplace=True)
ref_week_NET_NEW.drop_duplicates(subset='IID', keep='first', inplace=True)
qc_af = len(ref_week_NET_NEW)
print("Records Dropped : ",qc_bef-qc_af)

ref_week_NET_NEW = ref_week_NET_NEW.sort_values(['IID'])
ref_week_NET_NEW.reset_index(drop=True, inplace=True)

Records Dropped :  0


In [45]:
# EXPORT - 
ref_week_NET_NEW.to_parquet(f's3://vortex-staging-a65ced90/PYADM/weekly/archive/{CUR_WK}/reference/{net_new_varname}.parquet')

# QC


In [46]:
#17.1: Check current week's net_new source
# Perform the frequency count
freq_counts = ref_week_NET_NEW['SOURCE'].value_counts().reset_index()

# Rename the columns
freq_counts.columns = ['SOURCE', 'COUNT']

# Sort the frequencies in descending order
freq_counts = freq_counts.sort_values('SOURCE')

# Print the frequency counts
print(freq_counts)

     SOURCE  COUNT
6  20240503   1503
4  20240510   1523
1  20240517   1650
2  20240524   1626
0  20240531   1895
7  20240607   1330
3  20240614   1561
5  20240621   1519


In [47]:
#17.2: Check IC_INCL_LIN - this wont work yet because flag is not there (match code not avilable)
freq_counts = ref_week_NET_NEW['IC_INCL_LIN'].value_counts().reset_index()

# Rename the columns
freq_counts.columns = ['IC_INCL_LIN', 'COUNT']

# Sort the frequencies in descending order
freq_counts = freq_counts.sort_values('COUNT', ascending=False)

# Print the frequency counts #NOTE : last check on 27-12-2023 , 16 people with IC flag 0 are lesser in python compared to sas presumably Jami ppl
print(freq_counts)

   IC_INCL_LIN  COUNT
0            1   8059
1            0   4548


In [48]:
# 17.3: Check Net New for Dups
print("hcp_count",ref_week_NET_NEW['IID'].count())
print("distinct_hcp_count",ref_week_NET_NEW['IID'].nunique())

hcp_count 12607
distinct_hcp_count 12607
